In [1]:
#imports
# from __future__ import division
import itertools
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np
import pandas as pd
import math
from sklearn import metrics
from random import randint
from matplotlib import style
import seaborn as sns
import acquire as a
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import MinMaxScaler

In [2]:
df = a.get_curriculum_logs_data()

In [3]:
df.shape

(847330, 14)

In [4]:
df.head()

,time,path,user_id,cohort_id,ip,id,name,slack,start_date,end_date,created_at,updated_at,deleted_at,program_id
date,,,,,,,,,,,,,,
2018-01-26,09:55:03,/,1,8.0,97.105.19.61,8,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,NaN,1
2018-01-26,09:56:02,java-ii,1,8.0,97.105.19.61,8,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,NaN,1
2018-01-26,09:56:05,java-ii/object-oriented-programming,1,8.0,97.105.19.61,8,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,NaN,1
2018-01-26,09:56:06,slides/object_oriented_programming,1,8.0,97.105.19.61,8,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,NaN,1
2018-01-26,09:56:24,javascript-i/conditionals,2,22.0,97.105.19.61,22,Teddy,#teddy,2018-01-08,2018-05-17,2018-01-08 13:59:10,2018-01-08 13:59:10,NaN,2


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 847330 entries, 2018-01-26 to 2021-04-21
Data columns (total 14 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   time        847330 non-null  object 
 1   path        847329 non-null  object 
 2   user_id     847330 non-null  int64  
 3   cohort_id   847330 non-null  float64
 4   ip          847330 non-null  object 
 5   id          847330 non-null  int64  
 6   name        847330 non-null  object 
 7   slack       847330 non-null  object 
 8   start_date  847330 non-null  object 
 9   end_date    847330 non-null  object 
 10  created_at  847330 non-null  object 
 11  updated_at  847330 non-null  object 
 12  deleted_at  0 non-null       float64
 13  program_id  847330 non-null  int64  
dtypes: float64(2), int64(3), object(9)
memory usage: 97.0+ MB


# Prepare

 - change date to datetime object
 - set date as index
 - change start date and end date to datetime
 - rename path to endpoint
 - drop id column
 - rename name to cohort
 - maybe combine date and time? 

In [6]:
df.index = pd.to_datetime(df.index)

In [7]:
df.head()

,time,path,user_id,cohort_id,ip,id,name,slack,start_date,end_date,created_at,updated_at,deleted_at,program_id
date,,,,,,,,,,,,,,
2018-01-26,09:55:03,/,1,8.0,97.105.19.61,8,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,NaN,1
2018-01-26,09:56:02,java-ii,1,8.0,97.105.19.61,8,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,NaN,1
2018-01-26,09:56:05,java-ii/object-oriented-programming,1,8.0,97.105.19.61,8,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,NaN,1
2018-01-26,09:56:06,slides/object_oriented_programming,1,8.0,97.105.19.61,8,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,NaN,1
2018-01-26,09:56:24,javascript-i/conditionals,2,22.0,97.105.19.61,22,Teddy,#teddy,2018-01-08,2018-05-17,2018-01-08 13:59:10,2018-01-08 13:59:10,NaN,2


In [8]:
df.index

DatetimeIndex(['2018-01-26', '2018-01-26', '2018-01-26', '2018-01-26',
               '2018-01-26', '2018-01-26', '2018-01-26', '2018-01-26',
               '2018-01-26', '2018-01-26',
               ...
               '2021-04-21', '2021-04-21', '2021-04-21', '2021-04-21',
               '2021-04-21', '2021-04-21', '2021-04-21', '2021-04-21',
               '2021-04-21', '2021-04-21'],
              dtype='datetime64[ns]', name='date', length=847330, freq=None)

In [9]:
df.head()

,time,path,user_id,cohort_id,ip,id,name,slack,start_date,end_date,created_at,updated_at,deleted_at,program_id
date,,,,,,,,,,,,,,
2018-01-26,09:55:03,/,1,8.0,97.105.19.61,8,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,NaN,1
2018-01-26,09:56:02,java-ii,1,8.0,97.105.19.61,8,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,NaN,1
2018-01-26,09:56:05,java-ii/object-oriented-programming,1,8.0,97.105.19.61,8,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,NaN,1
2018-01-26,09:56:06,slides/object_oriented_programming,1,8.0,97.105.19.61,8,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,NaN,1
2018-01-26,09:56:24,javascript-i/conditionals,2,22.0,97.105.19.61,22,Teddy,#teddy,2018-01-08,2018-05-17,2018-01-08 13:59:10,2018-01-08 13:59:10,NaN,2


In [10]:
df.start_date = pd.to_datetime(df.start_date)
df.end_date = pd.to_datetime(df.end_date)

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 847330 entries, 2018-01-26 to 2021-04-21
Data columns (total 14 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   time        847330 non-null  object        
 1   path        847329 non-null  object        
 2   user_id     847330 non-null  int64         
 3   cohort_id   847330 non-null  float64       
 4   ip          847330 non-null  object        
 5   id          847330 non-null  int64         
 6   name        847330 non-null  object        
 7   slack       847330 non-null  object        
 8   start_date  847330 non-null  datetime64[ns]
 9   end_date    847330 non-null  datetime64[ns]
 10  created_at  847330 non-null  object        
 11  updated_at  847330 non-null  object        
 12  deleted_at  0 non-null       float64       
 13  program_id  847330 non-null  int64         
dtypes: datetime64[ns](2), float64(2), int64(3), object(7)
memory usage: 97.0+ MB


In [12]:
df = df.rename(columns={'path':'endpoint',
                       'name':'cohort'})

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 847330 entries, 2018-01-26 to 2021-04-21
Data columns (total 14 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   time        847330 non-null  object        
 1   endpoint    847329 non-null  object        
 2   user_id     847330 non-null  int64         
 3   cohort_id   847330 non-null  float64       
 4   ip          847330 non-null  object        
 5   id          847330 non-null  int64         
 6   cohort      847330 non-null  object        
 7   slack       847330 non-null  object        
 8   start_date  847330 non-null  datetime64[ns]
 9   end_date    847330 non-null  datetime64[ns]
 10  created_at  847330 non-null  object        
 11  updated_at  847330 non-null  object        
 12  deleted_at  0 non-null       float64       
 13  program_id  847330 non-null  int64         
dtypes: datetime64[ns](2), float64(2), int64(3), object(7)
memory usage: 97.0+ MB


In [14]:
df = df.drop(columns=['id'])

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 847330 entries, 2018-01-26 to 2021-04-21
Data columns (total 13 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   time        847330 non-null  object        
 1   endpoint    847329 non-null  object        
 2   user_id     847330 non-null  int64         
 3   cohort_id   847330 non-null  float64       
 4   ip          847330 non-null  object        
 5   cohort      847330 non-null  object        
 6   slack       847330 non-null  object        
 7   start_date  847330 non-null  datetime64[ns]
 8   end_date    847330 non-null  datetime64[ns]
 9   created_at  847330 non-null  object        
 10  updated_at  847330 non-null  object        
 11  deleted_at  0 non-null       float64       
 12  program_id  847330 non-null  int64         
dtypes: datetime64[ns](2), float64(2), int64(2), object(7)
memory usage: 90.5+ MB


# Test the Prep

In [ ]:
df = prep_logs()